In [ ]:
# if you use Google Colab
from google.colab import drive
drive.mount('/content/drive')

## Data Preperation

In [ ]:
def replace_split_to_tab (readed_file_path, written_file_path):
  read_text = open(readed_file_path).read()
  new_text = read_text.replace(' ||| ', '\t')
  open(written_file_path, 'w').write(new_text)

In [ ]:
readed_file_path = 'path_to/data/train/atis.uw.train.txt'
written_file_path = 'path_to/data/train/atis.uw.train.tabed.txt'
replace_split_to_tab(readed_file_path, written_file_path)

In [ ]:
_readed_file_path = 'path_to/data/test/atis.uw.test.txt'
written_file_path = 'path_to/data/test/atis.uw.test.tabed.txt'
replace_split_to_tab(readed_file_path, written_file_path)

In [ ]:
with open(written_file_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')

In [ ]:
lines[155]

"what airline is as\tSELECT DISTINCT airline.airline_code FROM airline WHERE airline.airline_code = 'AS'"

In [ ]:
len(lines)

448

In [ ]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [ ]:
num_samples = 10000
num_samples

10000

In [ ]:
for line in lines[: min(num_samples, len(lines) - 1)]:
  input_text, target_text = line.split('\t')
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [ ]:
input_texts[155]

'what airline is as'

In [ ]:
target_texts[155]

"\tSELECT DISTINCT airline.airline_code FROM airline WHERE airline.airline_code = 'AS'\n"

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 447
Number of unique input tokens: 40
Number of unique output tokens: 73
Max sequence length for inputs: 161
Max sequence length for outputs: 2990


In [ ]:
print(input_characters)

[' ', "'", '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
print(target_characters)

['\t', '\n', ' ', "'", '(', ')', '*', ',', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '<', '=', '>', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'y']


In [ ]:
input_token_index = dict(
  [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
  [(char, i) for i, char in enumerate(target_characters)])

In [ ]:
import numpy as np

encoder_input_data = np.zeros(
  (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
  dtype='float32')
decoder_input_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')
decoder_target_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')

In [ ]:
encoder_input_data.shape

(447, 161, 40)

In [ ]:
decoder_input_data.shape

(447, 2990, 73)

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t, target_token_index[char]] = 1.
    if t > 0:
      # decoder_target_data will be ahead by one timestep
      # and will not include the start character.
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [ ]:
encoder_input_data[155].shape

(161, 40)

## Building the Model

In [ ]:
import keras, tensorflow
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [ ]:
batch_size = 64  # batch size for training
epochs = 100  # number of epochs to train for
latent_dim = 256  # latent dimensionality of the encoding space

In [ ]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = Model(inputs=[encoder_inputs, decoder_inputs], 
              outputs=decoder_outputs)

## Training the Model

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 40)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 73)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 304128      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  337920      input_2[0][0]                    
                                                                 lstm_1[0][1]             

In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
model.save('path_to/results/seq2seq_text2sql.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 357 samples, validate on 90 samples
Epoch 1/100





357/357 [==============================] - 134s 377ms/step - loss: 0.7329 - val_loss: 0.7272
Epoch 2/100
357/357 [==============================] - 129s 360ms/step - loss: 0.6291 - val_loss: 0.7104
Epoch 3/100
357/357 [==============================] - 130s 365ms/step - loss: 0.6181 - val_loss: 0.7041
Epoch 4/100
357/357 [==============================] - 131s 367ms/step - loss: 0.6131 - val_loss: 0.6968
Epoch 5/100
357/357 [==============================] - 131s 367ms/step - loss: 0.6065 - val_loss: 0.6881
Epoch 6/100
357/357 [==============================] - 130s 363ms/step - loss: 0.5958 - val_loss: 0.6877
Epoch 7/100
357/357 [==============================] - 130s 363ms/step - loss: 0.5872 - val_loss: 0.6440
Epoch 8/100
357/357 [==============================] - 130s 363ms/step - loss: 0.5655 - val_loss: 0.6412
Epoch 9/100
3

## Testing the Model

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.load_weights('path_to/results/seq2seq_text2sql.h5')

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
  decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
  [decoder_inputs] + decoder_states_inputs,
  [decoder_outputs] + decoder_states)

In [ ]:
# reverse-lookup token index to turn sequences back to characters
reverse_input_char_index = dict(
  (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
  (i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
  # encode the input sequence to get the internal state vectors.
  states_value = encoder_model.predict(input_seq)
  
  # generate empty target sequence of length 1 with only the start character
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  target_seq[0, 0, target_token_index['\t']] = 1.
  
  # output sequence loop
  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict(
      [target_seq] + states_value)
    
    # sample a token and add the corresponding character to the 
    # decoded sequence
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += sampled_char
    
    # check for the exit condition: either hitting max length
    # or predicting the 'stop' character
    if (sampled_char == '\n' or 
        len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True
      
    # update the target sequence (length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.
    
    # update states
    states_value = [h, c]
    
  return decoded_sentence

In [ ]:
for seq_index in range(10):
  input_seq = encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print('-')
  print('Input sentence:', input_texts[seq_index])
  print('Decoded sentence:', decoded_sentence)

-
Input sentence: what flights go from dallas to phoenix
Decoded sentence: SELECT DISTINCT flight.flight_id FROM flight WHERE ( flight.from_airport IN ( SELECT airport_service.airport_code FROM airport_service WHERE airport_service.city_code IN ( SELECT city.city_code FROM city WHERE city.city_name = 'SALT AND ISELT )) AND ( flight.to_airport IN ( SELECT airport_service.airport_code FROM airport_service WHERE airport_service.city_code IN ( SELECT city.city_code FROM city WHERE city.city_name = 'SALT AND ISELT )) AND ( flight.to_airport IN ( SELECT airport_service.airport_code FROM airport_service WHERE airport_service.city_code IN ( SELECT city.city_code FROM city WHERE city.city_name = 'SALT AND ISELT )) AND ( flight.to_airport IN ( SELECT airport_service.airport_code FROM airport_service WHERE airport_service.city_code IN ( SELECT city.city_code FROM city WHERE city.city_name = 'SALT AND ISELT )) AND ( flight.to_airport IN ( SELECT airport_service.airport_code FROM airport_service WH

In [ ]:
input_sentence = "find me a flight from cincinnati to any airport in the new york city area"
test_sentence_tokenized = np.zeros(
  (1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
for t, char in enumerate(input_sentence):
  test_sentence_tokenized[0, t, input_token_index[char]] = 1.
print(input_sentence)
print(decode_sequence(test_sentence_tokenized))

find me a flight from cincinnati to any airport in the new york city area
SELECT DISTINCT flight.flight_id FROM flight WHERE ( flight.from_airport IN ( SELECT airport_service.airport_code FROM airport_service WHERE airport_service.city_code IN ( SELECT city.city_code FROM city WHERE city.city_name = 'SALT AND ISELT )) AND ( flight.to_airport IN ( SELECT airport_service.airport_code FROM airport_service WHERE airport_service.city_code IN ( SELECT city.city_code FROM city WHERE city.city_name = 'SALT AND ISELT )) AND ( flight.to_airport IN ( SELECT airport_service.airport_code FROM airport_service WHERE airport_service.city_code IN ( SELECT city.city_code FROM city WHERE city.city_name = 'SALT AND ISELT )) AND ( flight.to_airport IN ( SELECT airport_service.airport_code FROM airport_service WHERE airport_service.city_code IN ( SELECT city.city_code FROM city WHERE city.city_name = 'SALT AND ISELT )) AND ( flight.to_airport IN ( SELECT airport_service.airport_code FROM airport_service WHE

## Evaluation

In [ ]:
def load_lined_text(file_path):
    text = open(file_path, encoding='utf-8').read().split('\n')
    return text

In [ ]:
test_data = load_lined_text('path_to/data/test/atis.uw.test.tabed.txt')

In [ ]:
val_input_texts = []
val_target_texts = []
#line_ix = 12000
for line in test_data:
  try:
    input_text, target_text = line.split('\t')
    val_input_texts.append(input_text)
    val_target_texts.append(target_text)
  except:
    continue

val_encoder_input_data = np.zeros(
  (len(val_input_texts), max([len(txt) for txt in val_input_texts]),
   num_encoder_tokens), dtype='float32')

for i, input_text in enumerate(val_input_texts):
  for t, char in enumerate(input_text):
    val_encoder_input_data[i, t, input_token_index[char]] = 1.

In [ ]:
for seq_index in range(10):
  input_seq = val_encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print('-')
  print('Input sentence:', val_input_texts[seq_index])
  print('Decoded sentence:', decoded_sentence[:-1])
  print('Ground Truth sentence:', val_target_texts[seq_index])

-
Input sentence: what flights go from dallas to phoenix
Decoded sentence: SELECT DISTINCT flight.flight_id FROM flight WHERE ( flight.from_airport IN ( SELECT airport_service.airport_code FROM airport_service WHERE airport_service.city_code IN ( SELECT city.city_code FROM city WHERE city.city_name = 'SALT AND ISELT )) AND ( flight.to_airport IN ( SELECT airport_service.airport_code FROM airport_service WHERE airport_service.city_code IN ( SELECT city.city_code FROM city WHERE city.city_name = 'SALT AND ISELT )) AND ( flight.to_airport IN ( SELECT airport_service.airport_code FROM airport_service WHERE airport_service.city_code IN ( SELECT city.city_code FROM city WHERE city.city_name = 'SALT AND ISELT )) AND ( flight.to_airport IN ( SELECT airport_service.airport_code FROM airport_service WHERE airport_service.city_code IN ( SELECT city.city_code FROM city WHERE city.city_name = 'SALT AND ISELT )) AND ( flight.to_airport IN ( SELECT airport_service.airport_code FROM airport_service WH